## 分类声音

In [90]:
import torch

def load_data():
    import os
    import torchaudio

    
    xs = []
    ys = []
    
    for filename in os.listdir('data/superb'):
        if not filename.endswith('.wav'):
            continue
            
        x = torchaudio.load('data/superb/%s' % filename)[0]
        x = x.reshape(-1,1)
            
        y = int(filename[0])
        
        xs.append(x)
        ys.append(y)
        
    return xs,ys

xs,ys = load_data()

len(xs), len(ys), xs[0].shape, ys[0]

RuntimeError: Couldn't find appropriate backend to handle uri data/superb/1_20905.wav and format None.

In [ ]:
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size = 8,
                                     shuffle = True,
                                     drop_list = True)
x,y = next(item(loader))

len(loader),x.shape,y

In [29]:
#RNN神经网络层
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        
        #循环层
        self.rnn1 = torch.nn.RNN(input_size=1,hidden_size=16,batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=16,hidden_size=32,batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=32,hidden_size=64,batch_first=True)
        self.rnn4 = torch.nn.RNN(input_size=64,hidden_size=128,batch_first=True)
        
        #激活函数
        self.relu = torch.nn.ReLU()
        
        #池化层
        self.pool = torch.nn.AvgPool1d(kernel_size=7,stride=5)
        
        #线性输出
        self.fc = torch.nn.Linear(in_features=640,out_features=10)
        
    def forward(self,x):
        
        x,_ = self.rnn1(x)
        x = self.relu(x)
        
        x = x.permute(0,2,1)
        x = self.pool(x)
        x = x.permute(0,2,1)
        
        x,_ = self.rnn2(x)
        x = self.relu(x)
        x = x.permute(0,2,1)
        x = self.pool(x)
        x = x.permute(0,2,1)
        
        x, _ =self.rnn3(x)
        x = self.relu(x)
        x = x.permute(0,2,1)
        x = self.pool(x)
        x = x.permute(0,2,1)
        
        x, _ =self.rnn4(x)
        x = self.relu(x)
        x = x.permute(0,2,1)
        x = self.pool(x)
        x = x.permute(0,2,1)
        
        x = x.flatten(start_dim=1)
        
        return self.fc(x)
    
model = Model()

model(torch.randn(8,4000,1)).shape

torch.Size([8, 40])

In [31]:
def train():
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
    loss_fun = torch.nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(5):
        for i, (x,y) in enumerate(train_loader):
            out = model(x)
            loss = loss_fun(out,y)
            
            loss.backward
            optimizer.step()
            optimizer.zero_grad()
            
            if i%1000 == 0:
                acc = (out.argmax(dim=1)==y).sum().item()/len(y)
                print(epoch,i,loss.item(),acc)
                    
    torch.save(model,'model3.model')
    
train()

RuntimeError: Couldn't find appropriate backend to handle uri ./data_audio\SpeechCommands\speech_commands_v0.02\three\a04817c2_nohash_4.wav and format None.